In [3]:
%load_ext autoreload
%autoreload 2
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [4]:
import sys, os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
sys.path.insert(0, "..")
import jkinpylib
import meshcat
import torch
import numpy as np
import time
from time import sleep
torch.set_default_device(jkinpylib.config.DEVICE)

In [5]:
class Capsule():
    def __init__(self, p1, p2, r):
        super(Capsule, self).__init__()
        self.length = np.linalg.norm(p2 - p1)
        self.radius = r
        self.T = np.eye(4)
        self.T[:3, 3] = (p1 + p2) / 2
        v = p2 - p1
        v = v / np.linalg.norm(v)
        x, y, z = v
        sign = 1 if z > 0 else -1
        a = -1 / (sign + z)
        b = x * y * a
        t1 = np.array([1 + sign * x * x * a, sign * b, -sign * x])
        t2 = np.array([b, sign + y * y * a, -y])
        self.T[:3, 0] = t1
        self.T[:3, 1] = v # Meshcat uses y as axis of rotational symmetry
        self.T[:3, 2] = t2

In [6]:
SHOW_IN_JUPYTER = False

In [7]:
vis = meshcat.Visualizer()
robot = jkinpylib.robots.Panda()

# A bit hacky, must match the number of links in the robot
links = [
    "link0",
    "link1",
    "link2",
    "link3",
    "link4",
    "link5",
    "link6",
    "link7",
    # None,
    #"hand"
]
link_mask = torch.tensor([link is not None for link in links], dtype=torch.bool)

Tcube = meshcat.transformations.translation_matrix([0.4, 0.1, 0.5]) @ meshcat.transformations.rotation_matrix(0.7, [1, 2, 3])
cube_lengths = np.array([0.4, 0.5, 0.3])
vis[f"cuboid"].set_object(
    meshcat.geometry.Box(cube_lengths),
    meshcat.geometry.MeshToonMaterial(color=0xff8800, wireframe=True)
)
vis[f"cuboid"].set_transform(Tcube)

for link_i, link in enumerate(links):
    capsule = robot._collision_capsules[link_i, :].cpu().numpy().astype(np.float64)
    p1, p2, capsule_radius = capsule[0:3], capsule[3:6], capsule[6]
    capsule_geom = Capsule(p1, p2, capsule_radius)
    capsule_material = meshcat.geometry.MeshToonMaterial(color=0x8888ff, opacity=0.4)

    vis[f"{robot.name}/{link}/capsule/p1"].set_transform(meshcat.transformations.translation_matrix(p1))
    vis[f"{robot.name}/{link}/capsule/p2"].set_transform(meshcat.transformations.translation_matrix(p2))
    vis[f"{robot.name}/{link}/capsule/cyl"].set_transform(capsule_geom.T)
    vis[f"{robot.name}/{link}/mesh"].set_object(
        meshcat.geometry.DaeMeshGeometry.from_file(f"../jkinpylib/urdfs/panda/meshes/visual/{link}.dae"),
        meshcat.geometry.MeshLambertMaterial(color=0xffffff)
    )
    vis[f"{robot.name}/{link}/capsule/p1"].set_object(meshcat.geometry.Sphere(capsule_radius), capsule_material)
    vis[f"{robot.name}/{link}/capsule/p2"].set_object(meshcat.geometry.Sphere(capsule_radius), capsule_material)
    cyl_geom = meshcat.geometry.Cylinder(capsule_geom.length, capsule_radius)
    vis[f"{robot.name}/{link}/capsule/cyl"].set_object(cyl_geom, capsule_material)


if SHOW_IN_JUPYTER:
    vis.jupyter_cell()
else:
    vis.open()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
WorldModel::LoadRobot: /Users/drm/.cache/jkinpylib/temp_urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
Geometry: Loading 12 meshes from /Users/drm/Projects/IKFLow/jkinpylib/scripts/../jkinpylib/urdfs/panda/meshes/visual/link0.dae into Group
Geometry: Loading 4 meshes from /Users/drm/Projects/IKFLow/jkinpylib/scripts/../jkinpylib/urdfs/panda/meshes/visual/link3.dae into Group
Geometry: Loading 4 meshes from /Users/drm/Projects/IKFLow/jkinpylib/scripts/../jkinpylib/urdfs/panda/meshes/visual/link4.dae into Group
Geometry: Loading 3 meshes from /Users/drm/Projects/IKFLow/jkinpylib/scripts/../jkinpylib/urdfs/panda/meshes/visual/link5.dae into Group
Geometry: Loading 17 meshes from /Users/drm/Projects/IKFLow/jkinpylib/scripts/../jkinpyli

In [20]:
q = torch.zeros((1, 7))
n = robot._capsule_idx_to_link_idx.shape[0]


for t in range(200):

    joint_idx = t % 7
    l, u = robot.actuated_joints_limits[joint_idx]
    q[0, joint_idx] = (u-l)*torch.rand(1) + l
    q = robot.clamp_to_joint_limits(q)
    
    
    base_T_links = robot.forward_kinematics_batch(q, return_full_link_fk=True, out_device=q.device, dtype=q.dtype)

    T1s = base_T_links[:, robot._collision_idx0, :, :].reshape(-1, 4, 4)
    T2s = base_T_links[:, robot._collision_idx1, :, :].reshape(-1, 4, 4)
    c1s = robot._collision_capsules[robot._collision_idx0, :].expand(1, -1, -1).reshape(-1, 7)
    c2s = robot._collision_capsules[robot._collision_idx1, :].expand(1, -1, -1).reshape(-1, 7)
    start = time.time()
    self_dists = jkinpylib.geometry.capsule_capsule_distance_batch(c1s, T1s, c2s, T2s).reshape(1, -1)
    end = time.time()
    
    caps = robot._collision_capsules
    Tcaps = base_T_links[:, robot._capsule_idx_to_link_idx, :, :].reshape(-1, 4, 4)
    x, y, z = cube_lengths.astype(np.float32)/2
    cubes = torch.tensor([[-x, -y, -z, x, y, z]]).expand(n, 6)
    Tcubes = torch.tensor(Tcube, dtype=torch.float32).expand(n, 4, 4)
    start = time.time()
    env_dists = jkinpylib.geometry.capsule_cuboid_distance_batch(caps, Tcaps, cubes, Tcubes)
    end = time.time()

    is_self_collision = False
    for link_i, link in enumerate(links):
        if link is None:
            continue

        T = base_T_links[0, link_i, :, :].cpu().numpy().astype(np.float64)
        vis[f"{robot.name}/{link}"].set_transform(T)

        color = [0.0, 1.0, 0.0, 0.4]
        is_self_collide = torch.any(self_dists[0, robot._collision_idx0 == link_i] < 0) or torch.any(self_dists[0, robot._collision_idx1 == link_i] < 0)
        if is_self_collide:
            color = [1.0, 0.0, 0.0, 0.4]
            is_self_collision = True

        is_env_collide = env_dists[link_i, 0] < 0
        if is_env_collide:
            color = [1.0, 0.5, 0.0, 0.4]

        vis[f"{robot.name}/{link}/capsule/p1"].set_property("color", color)
        vis[f"{robot.name}/{link}/capsule/p2"].set_property("color", color)
        vis[f"{robot.name}/{link}/capsule/cyl"].set_property("color", color)

In [9]:
x = torch.tensor([[0, -np.pi/4, 0, 0.86*-3*np.pi/4, 0, np.pi/2, 0]])

base_T_links = robot.forward_kinematics_batch(x, return_full_link_fk=True, out_device=x.device, dtype=x.dtype)

T1s = base_T_links[:, robot._collision_idx0, :, :].reshape(-1, 4, 4)
T2s = base_T_links[:, robot._collision_idx1, :, :].reshape(-1, 4, 4)
c1s = robot._collision_capsules[robot._collision_idx0, :].expand(1, -1, -1).reshape(-1, 7)
c2s = robot._collision_capsules[robot._collision_idx1, :].expand(1, -1, -1).reshape(-1, 7)
self_dists = jkinpylib.geometry.capsule_capsule_distance_batch(c1s, T1s, c2s, T2s).reshape(1, -1)

n = robot._capsule_idx_to_link_idx.shape[0]
print("base_T_links.shape:", base_T_links.shape)
print("n:", n)
caps = robot._collision_capsules
print("robot._capsule_idx_to_link_idx:", robot._capsule_idx_to_link_idx.shape, robot._capsule_idx_to_link_idx)
Tcaps = base_T_links[:, robot._capsule_idx_to_link_idx, :, :].reshape(-1, 4, 4)
x, y, z = cube_lengths.astype(np.float32)/2
cubes = torch.tensor([[-x, -y, -z, x, y, z]]).expand(n, 6)
Tcubes = torch.tensor(Tcube, dtype=torch.float32).expand(n, 4, 4)
print(cubes)
print(caps.shape, Tcaps.shape, cubes.shape, Tcubes.shape)
env_dists = jkinpylib.geometry.capsule_cuboid_distance_batch(caps, Tcaps, cubes, Tcubes)

for link_i, link in enumerate(links):
    if link is None:
        continue

    T = base_T_links[0, link_i, :, :].cpu().numpy().astype(np.float64)
    vis[f"{robot.name}/{link}"].set_transform(T)

    color = [0.0, 1.0, 0.0, 0.4]
    is_self_collide = torch.any(self_dists[0, robot._collision_idx0 == link_i] < 0) or torch.any(self_dists[0, robot._collision_idx1 == link_i] < 0)
    if is_self_collide:
        color = [1.0, 0.0, 0.0, 0.4]
    
    is_env_collide = env_dists[link_i, 0] < 0
    if is_env_collide:
        color = [1.0, 0.5, 0.0, 0.4]

    vis[f"{robot.name}/{link}/capsule/p1"].set_property("color", color)
    vis[f"{robot.name}/{link}/capsule/p2"].set_property("color", color)
    vis[f"{robot.name}/{link}/capsule/cyl"].set_property("color", color)

base_T_links.shape: torch.Size([1, 10, 4, 4])
n: 8
robot._capsule_idx_to_link_idx: torch.Size([8]) tensor([0, 1, 2, 3, 4, 5, 6, 7], device='mps:0')
tensor([[-0.2000, -0.2500, -0.1500,  0.2000,  0.2500,  0.1500],
        [-0.2000, -0.2500, -0.1500,  0.2000,  0.2500,  0.1500],
        [-0.2000, -0.2500, -0.1500,  0.2000,  0.2500,  0.1500],
        [-0.2000, -0.2500, -0.1500,  0.2000,  0.2500,  0.1500],
        [-0.2000, -0.2500, -0.1500,  0.2000,  0.2500,  0.1500],
        [-0.2000, -0.2500, -0.1500,  0.2000,  0.2500,  0.1500],
        [-0.2000, -0.2500, -0.1500,  0.2000,  0.2500,  0.1500],
        [-0.2000, -0.2500, -0.1500,  0.2000,  0.2500,  0.1500]],
       device='mps:0')
torch.Size([8, 7]) torch.Size([8, 4, 4]) torch.Size([8, 6]) torch.Size([8, 4, 4])
